# 환경 준비

In [ ]:
# konlpy 설치하기 ## https://data1000.tistory.com/33
!pip3 install jpype1==0.7.0
!pip3 install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# konlpy를 실행하기
import konlpy

In [ ]:
## pandas을 실행하기, 단 실행 별명을 pd로. pandas 6자 pd 2자
import pandas as pd

# 데이터 불러오기

In [ ]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
## 문서 ID로 실제 파일 불러오기
# https://drive.google.com/file/d/1nE5AEG5YqZUBjwN5rTXceiqt5M85dLs-/view?usp=sharing

rawdata_downloaded = drive.CreateFile({'id': '1nE5AEG5YqZUBjwN5rTXceiqt5M85dLs-'})
rawdata_downloaded.GetContentFile('rawdata.tsv')

In [ ]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('rawdata.tsv', sep="\t")

In [ ]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,id,type
0,Seey Kaak,중국은 전부 각자 다 짜개져야된다 독립 응원함
1,들어봐 리슨,한글은 중국어 발음기호라 해
2,지숑,아 진짜 문화 말고 미세먼지랑 코로나나 가져가라
3,핑,코로나 색히들 진짜 불쌍하다
4,유찬,솔직히 중국게임 광고 맞춤법도 어감도 이상한거 참 짜증나고 욕밖에 안나옴
...,...,...
6039,돼지저금통,허미 쉿벌
6040,변승환,17초전은 못참지
6041,Ef Lee,조회수 1이래
6042,이호연,진짜 한복은 선넘었지


# 형태소 분석

In [ ]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [ ]:
# "원본데이터"를 대상으로 "okt"를 사용해서 형태소 분석하기
## 대상텍스트 1, 분류 1  >>>> type(텍스트) 기준 정렬      대상텍스트 1, 분류0로 함  >>>> id 기준 정렬
## 유튜브댓글 분석 시 type 필요 없음  

저장공간  = []
for index, row in 원본데이터.iterrows(): # 원본데이터의 컬럼(열)을 불러와서 반복할 준비!
    대상텍스트 = row[1] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    # 존경하는 국민 여러분. 오늘 저는 대한민국의 제16대 대통령에 취임하기 위해 이...
    # 분류 = row[1] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    #노무현
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    #형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [ ]:
# "형태소" 변수의 저장 내용 확인
형태소

[('전임', 'Noun'),
 ('문화', 'Noun'),
 ('재청', 'Noun'),
 ('연구원', 'Noun'),
 ('으로서', 'Josa'),
 ('너무', 'Adverb'),
 ('좋은', 'Adjective'),
 ('영상', 'Noun'),
 ('입니다', 'Adjective'),
 ('.', 'Punctuation')]

In [ ]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,count
0,전임,Noun,1
1,문화,Noun,1
2,재청,Noun,1
3,연구원,Noun,1
4,으로서,Josa,1
5,너무,Adverb,1
6,좋은,Adjective,1
7,영상,Noun,1
8,입니다,Adjective,1
9,.,Punctuation,1


In [ ]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[      0       1  count
 0    중국    Noun      1
 1     은    Josa      1
 2    전부    Noun      1
 3    각자    Noun      1
 4     다  Adverb      1
 5  짜개져야    Verb      1
 6    된다    Verb      1
 7    독립    Noun      1
 8    응원    Noun      1
 9     함    Noun      1,       0     1  count
 0    한글  Noun      1
 1     은  Josa      1
 2   중국어  Noun      1
 3  발음기호  Noun      1
 4     라  Josa      1
 5     해  Noun      1,       0            1  count
 0     아  Exclamation      1
 1    진짜         Noun      1
 2    문화         Noun      1
 3    말고         Josa      1
 4  미세먼지         Noun      1
 5     랑         Josa      1
 6   코로나         Noun      1
 7     나         Josa      1
 8  가져가라         Verb      1,       0          1  count
 0   코로나       Noun      1
 1     색       Noun      1
 2     히     Adverb      1
 3     들       Verb      1
 4    진짜       Noun      1
 5  불쌍하다  Adjective      1,        0           1  count
 0    솔직히   Adjective      1
 1     중국        Noun      1
 2     게임       

In [ ]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,count
0,중국,Noun,1
1,은,Josa,1
2,전부,Noun,1
3,각자,Noun,1
4,다,Adverb,1
...,...,...,...
5,너무,Adverb,1
6,좋은,Adjective,1
7,영상,Noun,1
8,입니다,Adjective,1


In [ ]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "카운트"]

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,카운트
0,중국,Noun,1
1,은,Josa,1
2,전부,Noun,1
3,각자,Noun,1
4,다,Adverb,1
...,...,...,...
5,너무,Adverb,1
6,좋은,Adjective,1
7,영상,Noun,1
8,입니다,Adjective,1


In [ ]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사'])['카운트'].sum()

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소     품사         
*       Punctuation       2
,       Punctuation     957
-       Punctuation      22
.       Punctuation    3097
/       Punctuation       7
                       ... 
힘없어지니까  Adjective         1
힘없으면    Adjective         1
힘자랑     Noun              1
🇨🇳      Foreign           1
🇭🇰      Foreign           1
Name: 카운트, Length: 18382, dtype: int64

In [ ]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

,,카운트
형태소,품사,
*,Punctuation,2
",",Punctuation,957
-,Punctuation,22
.,Punctuation,3097
/,Punctuation,7
...,...,...
힘없어지니까,Adjective,1
힘없으면,Adjective,1
힘자랑,Noun,1


In [ ]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')